http://preview.d2l.ai/d2l-en/PR-1111/chapter_multilayer-perceptrons/underfit-overfit.html#linear-function-fitting-underfitting

In [1]:
from d2l import torch as d2l
import torch
from torch import nn
import numpy as np
import math

In [2]:
max_degree = 20  # Maximum degree of the polynomial
n_train, n_test = 100, 100  # Training and test dataset sizes
true_w = np.zeros(max_degree)  # Allocate lots of empty space
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])

features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)  # `gamma(n)` = (n-1)!
# Shape of `labels`: (`n_train` + `n_test`,)
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)

In [3]:
features[:2].T, poly_features[:2, :], labels[:2]

(array([[ 1.2588966 , -0.51948561]]),
 array([[ 1.00000000e+00,  1.25889660e+00,  7.92410329e-01,
          3.32520891e-01,  1.04652355e-01,  2.63492989e-02,
          5.52850714e-03,  9.94259838e-04,  1.56458792e-04,
          2.18850490e-05,  2.75510139e-06,  3.15307980e-07,
          3.30783455e-08,  3.20324744e-09,  2.88039809e-10,
          2.41741558e-11,  1.90204767e-12,  1.40851844e-13,
          9.85099489e-15,  6.52704422e-16],
        [ 1.00000000e+00, -5.19485607e-01,  1.34932648e-01,
         -2.33651896e-02,  3.03446992e-03, -3.15272690e-04,
          2.72966041e-05, -2.02574185e-06,  1.31542967e-07,
         -7.59274202e-09,  3.94432020e-10, -1.86274325e-11,
          8.06390257e-13, -3.22237025e-14,  1.19569640e-15,
         -4.14098049e-17,  1.34448735e-18, -4.10848135e-20,
          1.18572051e-21, -3.24191969e-23]]),
 array([5.66245101, 3.78218313]))

In [4]:
def evaluate_loss(net, data_iter, loss):  #@save
    """Evaluate the loss of a model on the given dataset."""
    metric = d2l.Accumulator(2)  # Sum of losses, no. of examples
    for X, y in data_iter:
        l = loss(net(X), y)
        metric.add(l.sum(), l.numel())
    return metric[0] / metric[1]

In [5]:
def train(train_features, test_features, train_labels, test_labels,
          num_epochs=400):
    loss = nn.MSELoss()
    input_shape = train_features.shape[-1]
    # Switch off the bias since we already catered for it in the polynomial
    # features
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array((train_features, train_labels.reshape(-1,1)),
                                batch_size)
    test_iter = d2l.load_array((test_features, test_labels.reshape(-1,1)),
                               batch_size, is_train=False)
    trainer = torch.optim.SGD(net.parameters(), lr=0.01)
    animator = d2l.Animator(xlabel='epoch', ylabel='loss', yscale='log',
                            xlim=[1, num_epochs], ylim=[1e-3, 1e2],
                            legend=['train', 'test'])
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss),
                                     evaluate_loss(net, test_iter, loss)))
    print('weight:', net[0].weight.data.numpy())

In [6]:
# Pick the first four dimensions, i.e., 1, x, x^2/2!, x^3/3! from the
# polynomial features
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
      labels[:n_train], labels[n_train:])

TypeError: 'int' object is not callable

In [7]:
train(poly_features[:n_train, 0:4], poly_features[n_train:, 0:4],
      labels[:n_train].reshape(-1,1), labels[n_train:].reshape(-1,1))

TypeError: 'int' object is not callable